발전설비
- 지역별 설비 현황과 발전량의 상관성 파악
- 실제로 신재생 발전이 가능한 인프라가 얼마나 구축되었는지 그리고 현실적인 공급량 

In [5]:
import pandas as pd
#지역별 발전 설비
df = pd.read_csv('../Data/HOME_발전설비_연료원별.csv',encoding='cp949' )

print(df.head())


        기간 회원구분 급전방식 사업구분   지역     원자력     유연탄  무연탄     유류       LNG  ...  \
0      NaN  NaN  NaN  NaN  NaN     NaN     NaN  NaN    NaN       NaN  ...   
1  2025/04   합계   합계   합계   서울     0.0     0.0  0.0   0.00   802.346  ...   
2  2025/04   합계   합계   합계   부산  4550.0    19.0  0.0   0.00  1845.836  ...   
3  2025/04   합계   합계   합계   대구     0.0    72.9  0.0   0.00   640.700  ...   
4  2025/04   합계   합계   합계   인천     0.0  5080.0  0.0  36.23  8576.747  ...   

                  신재생 Unnamed: 12         Unnamed: 13 Unnamed: 14 Unnamed: 15  \
0                연료전지       석탄가스화                  태양          풍력          수력   
1   87.22999999999999           0   47.80411500000001           0           0   
2  40.480000000000004           0  275.20129999999995       0.762        0.02   
3                4.11           0   390.4740859999999           0        3.56   
4   265.1050000000001           0  181.41563499999995          49      12.599   

  Unnamed: 16 Unnamed: 17 Unnamed: 18     기타      

In [ ]:
#지역별 발전량

print(df.head())


     연도           서울            부산            대구            인천          광주  \
0  2023  5115143.000  3.749768e+07  2.132742e+06  4.819489e+07  840711.000   
1  2022  4336753.909  4.657933e+07  2.468916e+06  5.428345e+07  769032.947   
2  2021  5343889.297  4.035395e+07  2.811757e+06  6.050626e+07  644018.382   
3  2020  5122176.000  4.064738e+07  2.568644e+06  5.713563e+07  609551.000   
4  2019  1846891.377  3.752790e+07  2.877896e+06  6.005289e+07  562074.508   

           대전            울산            세종            경기            강원  \
0  303902.000  3.003559e+07  3.913222e+06  8.764708e+07  3.642853e+07   
1  295032.691  3.364133e+07  3.278703e+06  8.578016e+07  3.387653e+07   
2  182180.361  3.150358e+07  3.776644e+06  8.223265e+07  3.062279e+07   
3  167834.000  2.980867e+07  3.552567e+06  7.251266e+07  2.787505e+07   
4  167975.238  2.901605e+07  2.463630e+06  7.397688e+07  2.871589e+07   

            충북            충남            전북            전남            경북  \
0  3191549.000  1.